# Hyperparameter tuning with XGBoost, Ray Tune, Hyperopt and Optuna

## Introduction

In the post we are going to demonstrate how we can speed up hyperparameter tuning Bayesian optimization tuning algos like HyperOpt and Optuna. We'll also demonstrate the us of the Ray machine learning framework, which provides a unified API to many hyperparameter search algos, and lets us run them on a large cluster.

Outline:
- Overview of hyperparameter tuning
- Baseline linear regression with no hyperparameters
- ElasticNet with L1 and L2 regularization using ElasticNetCV hyperparameter optimization
- ElasticNet with GridSearchCV hyperparameter optimization
- XGBoost with sequential grid search over hyperparameter subsets with early stopping 
- XGBoost with Ray, HyperOpt and BayesOpt search algorithms
- HyperOpt on a Ray cluster
- Conclusions

But first, here are the results:

| ML Algo           | Hyperparameter search algo   | CV Error (RMSE in $)  | Time     |
|-------------------|------------------------------|-----------------------|----------|
| Linear Regression | None                         | $18192                |   0:01s  |
| ElasticNet        | ElasticNetCV (Grid Search)   | $18122                |   0:02s  |          
| ElasticNet        | GridSearchCV                 | $18061                |   0:05s  |          
| XGB               | Sequential Grid Search       | $18783                |   36:09  |
| XGB               | HyperOpt (128 samples)       | $18808                |   21:41  |
| XGB               | BayesOpt                     | $18506                | 1:15:04  |
| XGB               | Optuna                       | $18618

## Hyperparameter Tuning Overview

These are the main approaches to hyperparameter tuning https://en.wikipedia.org/wiki/Hyperparameter_optimization

- Grid search: given a finite set of discrete values for each hyperparameter, exhaustively cross-validate all combinations

- Random search: given a discrete or continuous distribution for each hyperparameter, randomly sample from the joint distribution. Generally [more efficient than exhaustive grid search.](https://dl.acm.org/doi/10.5555/2188385.2188395 ) 

- Bayesian optimization: update the search space as you search based on outcomes of prior searches

- Gradient-based optimization: attempt to estimate the gradient of the CV metric with respect to the hyperparameter and ascend/descend the gradient.

- Evolutionary optimization: sample the search space, discard combinations with poor metrics, and genetically evolve new combinations to try based on the successful combinations.

- Population-based: A method of performing hyperparameter optimization at the same time as training.

Below, we implement Bayesian optimization with HyperOpt and Optuna. When we perform a grid search, the search space can be considered a prior belief that the best hyperparameter vector is in the search space, and the combinations have equal probability of being the best combination. Perhaps we might do two passes. After an initial search on a broad, coarsely spaced grid, we might do a deeper dive in a smaller area around the best metric from the first pass, with a more finely-spaced grid. In Bayesian terminology we updated our prior belief.

Bayesian optimization first samples randomly, e.g. 30 combinations, and computes the cross-validation metric for each combination. Then it updates the distribution it samples from, so it is more likely to sample combinations near the good metrics, and less likely to sample combinations near the poor metrics. As it continues to sample, it continues to update the search distribution based on the metrics it finds.

Early stopping is also highly useful: often we can discard a combination without fully training it.

In this post we will use data from the Ames Housing Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques . The original data has 79 raw features. The data we will use has 100 features with a fair amount of feature engineering from [my own attempt at modeling](https://github.com/druce/iowa), which was in the top 5% or so when I submitted it.

Further reading: 
https://arxiv.org/abs/1502.02127
https://link.springer.com/chapter/10.1007/978-3-030-05318-5_1
https://arxiv.org/abs/2003.05689



In [1]:
from itertools import product
from datetime import datetime, timedelta
import os
import random
import string

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

#!conda install -y -c conda-forge  xgboost 
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

import lightgbm
from lightgbm import LGBMRegressor

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger

# pip install hyperopt
# pip install optuna

import wandb
os.environ['WANDB_NOTEBOOK_NAME']='hyperparameter_optimization.ipynb'

print(datetime.now())

print ("%-20s %s"% ("numpy", np.__version__))
print ("%-20s %s"% ("pandas", pd.__version__))
print ("%-20s %s"% ("sklearn", sklearn.__version__))
print ("%-20s %s"% ("xgboost", xgboost.__version__))
print ("%-20s %s"% ("lightgbm", lightgbm.__version__))
print ("%-20s %s"% ("ray", ray.__version__))


2020-10-13 14:21:50.338326
numpy                1.19.1
pandas               1.1.3
sklearn              0.23.2
xgboost              1.2.0
lightgbm             2.3.0
ray                  1.0.0


In [2]:
# set seed for reproducibility
RANDOMSTATE = 42
np.random.seed(RANDOMSTATE)


In [3]:
def get_random_tag(length):
    """random tag for experiments"""
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    return result_str.upper()

get_random_tag(8)

'8XOMPRI7'

In [4]:
# import train data
df = pd.read_pickle('df_train.pickle')

response = 'SalePrice'
predictors = ['YearBuilt',
              'BsmtFullBath',
              'FullBath',
              'KitchenAbvGr',
              'GarageYrBlt',
              'LotFrontage',
              'MasVnrArea',
              '1stFlrSF',
              'GrLivArea',
              'GarageArea',
              'WoodDeckSF',
              'PorchSF',
              'AvgBltRemod',
              'FireBathRatio',
              'TotalSF x OverallQual x OverallCond',
              'AvgBltRemod x Functional x TotalFinSF',
              'Functional x OverallQual',
              'KitchenAbvGr x KitchenQual',
              'GarageCars x GarageYrBlt',
              'GarageQual x GarageCond x GarageCars',
              'HeatingQC x Heating',
              'monthnum',
              'log_YearBuilt',
              'log_LotArea',
              'log_TotalFinSF',
              'log_GarageRatio',
              'log_TotalSF x OverallQual x OverallCond',
              'log_TotalSF x OverallCond',
              'log_AvgBltRemod x TotalFinSF',
              'sq_2ndFlrSF',
              'sq_BsmtFinSF',
              'sq_BsmtFinSF x BsmtQual',
              'sq_BsmtFinSF x BsmtBath',
              'BldgType_4',
              'BsmtExposure_1',
              'BsmtExposure_4',
              'BsmtFinType1_1',
              'BsmtFinType1_2',
              'BsmtFinType1_4',
              'BsmtFinType1_5',
              'BsmtFinType1_6',
              'CentralAir_0',
              'CentralAir_1',
              'Condition1_1',
              'Condition1_3',
              'ExterCond_2',
              'ExterQual_2',
              'Exterior1st_4',
              'Exterior1st_5',
              'Exterior1st_10',
              'Fence_0',
              'Fence_2',
              'Foundation_1',
              'Foundation_5',
              'GarageCars_1',
              'GarageFinish_2',
              'GarageFinish_3',
              'GarageType_2',
              'HouseStyle_2',
              'KitchenQual_4',
              'LotConfig_0',
              'LotConfig_4',
              'MSSubClass_30',
              'MSSubClass_70',
              'MSZoning_0',
              'MSZoning_1',
              'MSZoning_4',
              'MasVnrType_2',
              'MasVnrType_3',
              'MoSold_1',
              'MoSold_5',
              'MoSold_6',
              'MoSold_11',
              'Neighborhood_3',
              'Neighborhood_4',
              'Neighborhood_5',
              'Neighborhood_10',
              'Neighborhood_11',
              'Neighborhood_16',
              'Neighborhood_17',
              'Neighborhood_19',
              'Neighborhood_22',
              'Neighborhood_24',
              'OverallCond_7',
              'OverallQual_5',
              'OverallQual_6',
              'OverallQual_7',
              'OverallQual_9',
              'PavedDrive_0',
              'PavedDrive_2',
              'SaleCondition_1',
              'SaleCondition_2',
              'SaleCondition_5',
              'SaleType_4',
              'BedroomAbvGr_1',
              'BedroomAbvGr_4',
              'BedroomAbvGr_5',
              'HalfBath_1',
              'TotalBath_1.0',
              'TotalBath_2.5']

X_train, X_test, y_train, y_test = train_test_split(df, df[response], test_size=.25)

display(df[predictors].head())
display(df[[response]].head())


,YearBuilt,BsmtFullBath,FullBath,KitchenAbvGr,GarageYrBlt,LotFrontage,MasVnrArea,1stFlrSF,GrLivArea,GarageArea,...,SaleCondition_1,SaleCondition_2,SaleCondition_5,SaleType_4,BedroomAbvGr_1,BedroomAbvGr_4,BedroomAbvGr_5,HalfBath_1,TotalBath_1.0,TotalBath_2.5
Id,,,,,,,,,,,,,,,,,,,,,
1,7,1,2,1,7,65.0,196.0,856,1710,548.0,...,0,0,0,1,0,0,0,1,0,0
2,34,0,2,1,34,80.0,0.0,1262,1262,460.0,...,0,0,0,1,0,0,0,0,0,1
3,9,1,2,1,9,68.0,162.0,920,1786,608.0,...,0,0,0,1,0,0,0,1,0,0
4,95,1,1,1,12,60.0,0.0,961,1717,642.0,...,1,0,0,1,0,0,0,0,0,0
5,10,1,2,1,10,84.0,350.0,1145,2198,836.0,...,0,0,0,1,0,1,0,1,0,0


,SalePrice
Id,
1,12.247699
2,12.109016
3,12.317171
4,11.849405
5,12.429220


In [5]:
# we are training on a response which is the log of 1 + the sale price
# transform prediction back to original basis with expm1 and evaluate vs. original

def evaluate(y_train, y_pred_train, y_test, y_pred_test):
    """evaluate in train_test split"""
    print('Train RMSE', np.sqrt(mean_squared_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print('Train R-squared', r2_score(np.expm1(y_train), np.expm1(y_pred_train)))
    print('Train MAE', mean_absolute_error(np.expm1(y_train), np.expm1(y_pred_train)))
    print()
    print('Test RMSE', np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred_test))))
    print('Test R-squared', r2_score(np.expm1(y_test), np.expm1(y_pred_test)))
    print('Test MAE', mean_absolute_error(np.expm1(y_test), np.expm1(y_pred_test)))

MEAN_RESPONSE=df[response].mean()
def cv_to_raw(cv_val):
    """convert log1p rmse to underlying SalePrice error"""
    return np.expm1(MEAN_RESPONSE+cv_val) - np.expm1(MEAN_RESPONSE)

In [6]:
# always use same k-folds for reproducibility
kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOMSTATE)


## Baseline linear regression
- Raw CV RMSE 18191.9791
- Wall time 2.81 s

In [7]:
%%time
# Tune lr search space for alphas and l1_ratio
print("LinearRegression")

print(len(predictors), "predictors")

lr = LinearRegression()

#train and evaluate in train/test split
lr.fit(X_train[predictors], y_train)

y_pred_train = lr.predict(X_train[predictors])
y_pred_test = lr.predict(X_test[predictors])
evaluate(y_train, y_pred_train, y_test, y_pred_test)

# evaluate using kfolds, same process as train/test split but average results over 10 folds
# more sample-efficient, less CPU-efficient

scores = -cross_val_score(lr, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.04f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


LinearRegression
100 predictors
Train RMSE 16551.16362165147
Train R-squared 0.9554490130823184
Train MAE 10998.860388695648

Test RMSE 17846.17192179596
Test R-squared 0.9364301769277958
Test MAE 12755.395673639627

Log1p CV RMSE 0.1037 (STD 0.0099)
Raw CV RMSE 18191.9791 (STD 1838.6678)
CPU times: user 184 ms, sys: 189 ms, total: 373 ms
Wall time: 1.06 s


## Native Sklearn xxxCV
- LogisticRegressionCV, LassoCV, RidgeCV, ElasticNetCV, etc.
- Test many hyperparameters in parallel with multithreading
- Note improvement vs. LinearRegression due to controlling overfitting
- RMSE $18103
- Time 5s


In [8]:
%%time
# Tune elasticnet search space for alphas and L1_ratio
# predictor selection used to create the training set used lasso
# so l1 parameter is close to 0
# could use ridge (eg elasticnet with 0 L1 regularization)
# but then only 1 param, more general and useful to do this with elasticnet
print("ElasticnetCV")

# make pipeline
# with regularization must scale predictors
elasticnetcv = make_pipeline(RobustScaler(),
                             ElasticNetCV(max_iter=100000, 
                                          l1_ratio=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                          alphas=np.logspace(-4, -2, 9),
                                          cv=kfolds,
                                          n_jobs=-1,
                                          verbose=1,
                                         ))

#train and evaluate in train/test split
elasticnetcv.fit(X_train[predictors], y_train)

y_pred_train = elasticnetcv.predict(X_train[predictors])
y_pred_test = elasticnetcv.predict(X_test[predictors])
evaluate(y_train, y_pred_train, y_test, y_pred_test)
l1_ratio = elasticnetcv._final_estimator.l1_ratio_
alpha = elasticnetcv._final_estimator.alpha_
print('l1_ratio', l1_ratio)
print('alpha', alpha)

# evaluate using kfolds on full dataset
# I don't see API to get CV error from elasticnetcv, so we use cross_val_score
elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=10000)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.04f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


ElasticnetCV


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
.................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.8s
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Train RMSE 16782.09907754634
Train R-squared 0.954197115772813
Train MAE 11025.896226946616

Test RMSE 17457.333905669522
Test R-squared 0.9391701570474736
Test MAE 12389.820674779756
l1_ratio 0.01
alpha 0.005623413251903491

Log1p CV RMSE 0.1033 (STD 0.0112)
Raw CV RMSE 18122.0127 (STD 2074.9545)
CPU times: user 7.2 s, sys: 5.77 s, total: 13 s
Wall time: 2.1 s


## GridSearchCV
- Useful for algos with no native multithreaded xxxCV
- Test many hyperparameter combinations in parallel with multithreading
- Similar result vs ElasticNetCV, not exact, need more research as to why


In [9]:
%%time
gs = make_pipeline(RobustScaler(),
                   GridSearchCV(ElasticNet(max_iter=100000),
                                param_grid={'l1_ratio': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                            'alpha': np.logspace(-4, -2, 9),
                                           },
                                scoring='neg_mean_squared_error',
                                refit=True,
                                cv=kfolds,
                                n_jobs=-1,
                                verbose=1
                               ))

# do cv using kfolds on full dataset
print("\nCV on full dataset")
gs.fit(df[predictors], df[response])
print('best params', gs._final_estimator.best_params_)
print('best score', -gs._final_estimator.best_score_)
l1_ratio = gs._final_estimator.best_params_['l1_ratio']
alpha = gs._final_estimator.best_params_['alpha']

elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=100000)
print(elasticnet)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))

# difference in average CV scores reported by GridSearchCV and cross_val_score
# with same alpha, l1_ratio, kfolds
# one reason could be that we used simple average, GridSearchCV is weighted by # of samples per fold?
nsamples = [len(z[1]) for z in kfolds.split(df)]
print("weighted average %.06f" % np.average(scores, weights=nsamples))
# not sure why, also ElasticSearchCV shows fewer fits, takes less time



CV on full dataset
Fitting 10 folds for each of 117 candidates, totalling 1170 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1116 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1170 out of 1170 | elapsed:    4.0s finished


best params {'alpha': 0.0031622776601683794, 'l1_ratio': 0.01}
best score 0.010637685614240404
ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=100000)

Log1p CV RMSE 0.103003 (STD 0.0109)
Raw CV RMSE 18060.902698 (STD 2008.2407)
weighted average 0.103023
CPU times: user 1.28 s, sys: 399 ms, total: 1.68 s
Wall time: 4.45 s


In [10]:
# roll-our-own CV 
# matches cross_val_score
alpha = 0.0031622776601683794
l1_ratio = 0.01
regressor = ElasticNet(alpha=alpha,
                       l1_ratio=l1_ratio,
                       max_iter=10000)
print(regressor)
cverrors = []
for train_fold, cv_fold in kfolds.split(df): 
    fold_X_train=df[predictors].values[train_fold]
    fold_y_train=df[response].values[train_fold]
    fold_X_test=df[predictors].values[cv_fold]
    fold_y_test=df[response].values[cv_fold]
    regressor.fit(fold_X_train, fold_y_train)
    y_pred_test=regressor.predict(fold_X_test)
    cverrors.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
    
print("%.06f" % np.average(cverrors))
    

ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=10000)
0.103003


## XGBoost Sequential Grid Search CV 
- XGBoost has native multithreading, CV
- XGBoost has many tuning parameters so a complete grid search has an unreasonable number of combinations
- We tune reduced sets sequentially and use early stopping. 

### Tuning methodology
- Set an initial set of starting parameters
- Do 10-fold CV
- Use early stopping to halt training in each fold if no improvement after eg 100 rounds, pick hyperparameters to minimize average error over kfolds
- Tune sequentially on groups of hyperparameters that don't interact too much between groups to reduce combinations
- Tune max_depth and min_child_weight 
- Tune subsample and colsample_bytree
- Tune alpha, lambda and gamma (regularization)
- Tune learning rate: lower learning rate will need more rounds/n_estimators
- Retrain on full dataset with best learning rate and best n_estimators (average stopping point over kfolds)

### Notes
- It doesn't seem possible to get XGBoost early stopping and also use GridSearchCV. GridSearchCV doesn't pass the kfolds in a way that XGboost understands for early stopping
- 2 alternative approaches 
    - use native xgboost .cv which understands early stopping but doesn't use sklearn API (uses DMatrix, not np array or dataframe)
    - use sklearn API and roll our own grid search instead of GridSearchCV (used below)
- XGboost terminology differs from sklearn
    - boost_rounds = n_estimators
    - eta = learning_rate
- parameter reference: https://xgboost.readthedocs.io/en/latest/parameter.html
- training reference: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
- times are wall times on an amazon t2.2xlarge instance  
- to set up environment:
    - `conda create --name hyperparam python=3.8`
    - `conda activate hyperparam`
    - `conda install jupyter`
    - `pip install -r requirements.txt`
- round 1 Wall time: 6min 23s
- round 2 Wall time: 19min 22s
- round 3 Wall time: 5min 30s
- round 4 Wall time: 4min 54s
- total time 36:09
- RMSE 18783.117031

In [11]:
#%%time
# this cell runs a single round
# the full process is 
# set initial XGboost parameters
# remove overrides (search for TODO: in this cell)
# run round 1 and override initial max_depth, min_child_weight based on best values (search for TODO:)
# run round 2 and override subsample and colsample_bytree based on best values
# run round 3 and override reg_alpha, reg_lambda, reg_gamma based on best values
# run round 4 and obtain learning_rate and best n_iterations
# this is not an exhaustive list but a representative list of most important parameters to tune
# see https://xgboost.readthedocs.io/en/latest/parameter.html for all parameters

# in XGBoost > 1.0.2 this seems to give a warning
# Parameters: { early_stopping_rounds } might not be used.
# but early stopping seems to be used correctly

max_depth = 5
min_child_weight=5
colsample_bytree = 0.5
subsample = 0.5
reg_alpha = 1e-05
reg_lambda = 1
reg_gamma = 0
learning_rate = 0.01

BOOST_ROUNDS=50000   # we use early stopping so make this arbitrarily high
EARLY_STOPPING_ROUNDS=100 # stop if no improvement after 100 rounds

# round 1: tune depth and min_child_weight
max_depths = list(range(1,5))
min_child_weights = list(range(1,5))
gridsearch_params_1 = product(max_depths, min_child_weights)

# round 2: tune subsample and colsample_bytree
subsamples = np.linspace(0.1, 1.0, 10)
colsample_bytrees = np.linspace(0.1, 1.0, 10)
gridsearch_params_2 = product(subsamples, colsample_bytrees)

# round 2 (refined): tune subsample and colsample_bytree
subsamples = np.linspace(0.4, 0.8, 9)
colsample_bytrees = np.linspace(0.05, 0.25, 5)
gridsearch_params_2 = product(subsamples, colsample_bytrees)

# round 3: tune alpha, lambda, gamma
reg_alphas = np.logspace(-3, -2, 3)
reg_lambdas = np.logspace(-2, 1, 4)
reg_gammas = [0]
#reg_gammas = np.linspace(0, 5, 6)
gridsearch_params_3 = product(reg_alphas, reg_lambdas, reg_gammas)

# round 4: learning rate
learning_rates = reversed(np.logspace(-3, -1, 5).tolist())
gridsearch_params_4 = learning_rates

# TODO: remove these overrides to reset the search
# override initial parameters after search
# round 1:
max_depth=2
min_child_weight=2
# # round 2:
subsample=0.60
colsample_bytree=0.05
# # round 3:  
reg_alpha = 0.003162
reg_lambda = 0.1
reg_gamma = 0

def my_cv(df, predictors, response, kfolds, regressor, verbose=False):
    """Roll our own CV over kfolds with early stopping"""
    metrics = []
    best_iterations = []

    for train_fold, cv_fold in kfolds.split(df): 
        fold_X_train=df[predictors].values[train_fold]
        fold_y_train=df[response].values[train_fold]
        fold_X_test=df[predictors].values[cv_fold]
        fold_y_test=df[response].values[cv_fold]
        regressor.fit(fold_X_train, fold_y_train,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      eval_set=[(fold_X_test, fold_y_test)],
                      eval_metric='rmse',
                      verbose=verbose
                     )
        y_pred_test=regressor.predict(fold_X_test)
        metrics.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
        best_iterations.append(xgb.best_iteration)
    return np.average(metrics), np.std(metrics), np.average(best_iterations)

results = []
best_iterations = []

# TODO: iteratively uncomment 1 of the following 4 lines
# for i, (max_depth, min_child_weight) in enumerate(gridsearch_params_1): # round 1
# for i, (subsample, colsample_bytree) in enumerate(gridsearch_params_2): # round 2
# for i, (reg_alpha, reg_lambda, reg_gamma) in enumerate(gridsearch_params_3): # round 3
for i, learning_rate in enumerate(gridsearch_params_4): # round 4

    params = {
        'max_depth': max_depth,
        'min_child_weight': min_child_weight,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'gamma': reg_gamma,
        'learning_rate': learning_rate,
    }
    print("%s params  %3d: %s" % (datetime.strftime(datetime.now(), "%T"), i, params))
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=BOOST_ROUNDS,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        random_state=RANDOMSTATE,    
        verbosity=1,
        n_jobs=-1,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1,      
        **params
    )
    
    metric_rmse, metric_std, best_iteration = my_cv(df, predictors, response, kfolds, xgb, verbose=False)    
    results.append([max_depth, min_child_weight, subsample, colsample_bytree, reg_alpha, reg_lambda, reg_gamma, 
                   learning_rate, metric_rmse, metric_std, best_iteration])
    
    print("%s %3d result mean: %.6f std: %.6f, iter: %.2f" % (datetime.strftime(datetime.now(), "%T"), i, metric_rmse, metric_std, best_iteration))


results_df = pd.DataFrame(results, columns=['max_depth', 'min_child_weight', 'subsample', 'colsample_bytree', 
                               'reg_alpha', 'reg_lambda', 'reg_gamma', 'learning_rate', 'rmse', 'std', 'best_iter']).sort_values('rmse')
results_df


03:24:18 params    0: {'max_depth': 2, 'min_child_weight': 2, 'subsample': 0.6, 'colsample_bytree': 0.05, 'reg_alpha': 0.003162, 'reg_lambda': 0.1, 'gamma': 0, 'learning_rate': 0.1}
[03:24:19] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:24:19] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:24:20] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to 

[03:24:41] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:24:42] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:24:45] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if y

[03:27:49] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:28:09] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:28:29] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if y

,max_depth,min_child_weight,subsample,colsample_bytree,reg_alpha,reg_lambda,reg_gamma,learning_rate,rmse,std,best_iter
1,2,2,0.6,0.05,0.003162,0.1,0,0.031623,0.105996,0.012395,1134.8
2,2,2,0.6,0.05,0.003162,0.1,0,0.010000,0.106046,0.013608,2888.1
3,2,2,0.6,0.05,0.003162,0.1,0,0.003162,0.106838,0.013188,7783.7
4,2,2,0.6,0.05,0.003162,0.1,0,0.001000,0.107344,0.013132,22413.3
0,2,2,0.6,0.05,0.003162,0.1,0,0.100000,0.109786,0.010449,476.0


In [12]:
# get best hyperparameters from results dataframe
max_depth = int(results_df.iloc[0]['max_depth'])
min_child_weight = results_df.iloc[0]['min_child_weight']
subsample = results_df.iloc[0]['subsample']
colsample_bytree = results_df.iloc[0]['colsample_bytree']
reg_alpha = results_df.iloc[0]['reg_alpha']
reg_lambda = results_df.iloc[0]['reg_lambda']
reg_gamma = results_df.iloc[0]['reg_gamma']
learning_rate = results_df.iloc[0]['learning_rate']
N_ESTIMATORS = int(results_df.iloc[0]['best_iter'])

params = {
    'max_depth': int(max_depth),
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators': N_ESTIMATORS,    
}

print(params)

{'max_depth': 2, 'min_child_weight': 2.0, 'subsample': 0.6, 'colsample_bytree': 0.05, 'reg_alpha': 0.003162, 'reg_lambda': 0.1, 'gamma': 0.0, 'learning_rate': 0.03162277660168379, 'n_estimators': 1134}


In [26]:
%%time
# run from scratch early stopping and evaluate

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    booster='gbtree',   
    base_score=0.5, 
    scale_pos_weight=1,        
    **params
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.05, gamma=0.0,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.03162277660168379, max_delta_step=None,
             max_depth=2, min_child_weight=2.0, missing=nan,
             monotone_constraints=None, n_estimators=1134, n_jobs=-1,
             num_parallel_tree=None, random_state=42, reg_alpha=0.003162,
             reg_lambda=0.1, scale_pos_weight=1, subsample=0.6,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106893 (STD 0.0125)
Raw CV RMSE 18783.117031 (STD 2307.2858)
CPU times: user 89.6 ms, sys: 164 ms, total: 253 ms
Wall time: 3.52 s


In [8]:
# refactor for ray.tune
def my_xgb(config):
    
    # fix these configs 
    config['max_depth'] += 2   # hyperopt needs left to start at 0 but we want to start at 2
    config['max_depth'] = int(config['max_depth'])
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_jobs=1,
        random_state=RANDOMSTATE,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1, 
        **config,
    )
    scores = np.sqrt(-cross_val_score(xgb, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}


In [15]:
config = {
    'max_depth': max_depth-2,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators': N_ESTIMATORS,
}

xgb = my_xgb(config)

print(xgb)
# same mse

Session not detected. You should not be calling this function outside `tune.run` or while using the class API. 


{'mse': 0.10689314042904505}


## HyperOpt
https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf
https://github.com/hyperopt/hyperopt
http://hyperopt.github.io/hyperopt/
https://blog.dominodatalab.com/hyperopt-bayesian-hyperparameter-optimization/
    

In [16]:
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.randint(0, 6),
        'min_child_weight': tune.randint(0, 6),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,wandb/api_key_file,wandb/log_config,wandb/project,iter,total time (s),mse
my_xgb_47fe8acc,TERMINATED,,0.7,0,0.0105391,0,4,435.406,0.000188888,9.90358,0.65,secrets/wandb.txt,True,iowa,1,8.70135,0.203631
my_xgb_48012cf0,TERMINATED,,0.3,0,0.00673344,3,2,428.153,0.538855,2.32463,0.6,secrets/wandb.txt,True,iowa,1,6.20968,0.681993
my_xgb_48032aaa,TERMINATED,,0.15,0,0.00133237,5,4,7988.18,0.00163546,19.9476,0.8,secrets/wandb.txt,True,iowa,1,137.821,0.114781
my_xgb_4804db3e,TERMINATED,,0.2,0,0.0187659,2,3,735.64,0.000262686,32.5338,0.6,secrets/wandb.txt,True,iowa,1,11.6776,0.11581
my_xgb_480686c8,TERMINATED,,0.6,0,0.00155207,0,3,4032.08,0.0246711,0.00107456,0.8,secrets/wandb.txt,True,iowa,1,78.3048,0.124603
my_xgb_48083cf2,TERMINATED,,0.15,0,0.00106609,1,5,2700.39,0.0127785,0.0424791,0.8,secrets/wandb.txt,True,iowa,1,31.1441,0.661069
my_xgb_4809e5c0,TERMINATED,,0.4,0,0.00556109,3,2,238.15,0.000451262,43.2731,0.75,secrets/wandb.txt,True,iowa,1,2.93189,3.25644
my_xgb_480b78b8,TERMINATED,,0.55,0,0.0154611,0,1,5605.97,0.00272955,4.48938,0.7,secrets/wandb.txt,True,iowa,1,109.085,0.109036
my_xgb_480d03e0,TERMINATED,,0.5,0,0.00126552,5,3,4734.92,0.00967981,33.1173,0.65,secrets/wandb.txt,True,iowa,1,148.559,0.1519
my_xgb_480e8e68,TERMINATED,,0.75,0,0.00153686,2,2,678.644,0.306625,1.59856,0.9,secrets/wandb.txt,True,iowa,1,12.0086,4.08285


Start Time           2020-10-13 03:29:12.649826
End Time             2020-10-13 03:58:48.385875
0:29:35


In [17]:
analysis.results_df.columns

Index(['mse', 'time_this_iter_s', 'done', 'timesteps_total', 'episodes_total',
       'training_iteration', 'experiment_id', 'date', 'timestamp',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'timesteps_since_restore', 'iterations_since_restore', 'experiment_tag',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight',
       'config.subsample', 'config.colsample_bytree', 'config.reg_alpha',
       'config.reg_lambda', 'config.gamma', 'config.learning_rate',
       'config.wandb.project', 'config.wandb.api_key_file',
       'config.wandb.log_config'],
      dtype='object')

In [18]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df


,mse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.min_child_weight,config.subsample,config.colsample_bytree,config.reg_alpha,config.reg_lambda,config.gamma,config.learning_rate
trial_id,,,,,,,,,,,,
481b3802,0.106830,2020-10-13_03-31-04,52.612342,4374,4,3,0.75,0.10,0.009577,1.219649,0,0.023078
4930042a,0.106832,2020-10-13_03-36-27,88.308606,7221,3,3,0.65,0.15,0.013007,1.561307,0,0.009488
49355baa,0.106860,2020-10-13_03-35-43,43.248692,5468,3,4,0.80,0.05,0.072716,2.172529,0,0.007060
4deb0a78,0.107215,2020-10-13_03-57-51,47.863379,5982,3,4,0.60,0.05,0.028824,7.170552,0,0.009774
49d6c238,0.107370,2020-10-13_03-39-11,48.379306,6130,3,4,0.80,0.05,0.024101,5.281423,0,0.012514
...,...,...,...,...,...,...,...,...,...,...,...,...
497b08b2,6.709540,2020-10-13_03-36-51,7.879984,213,6,5,0.80,0.65,0.230677,0.000495,0,0.002540
4c0fad4e,6.890884,2020-10-13_03-49-00,12.949771,320,5,5,0.80,0.70,0.136117,0.000365,0,0.001608
4a40eb68,7.572926,2020-10-13_03-40-51,6.827069,203,6,5,0.80,0.70,0.331362,0.000547,0,0.002070


In [19]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    


In [20]:
best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.1, gamma=0, gpu_id=None,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.023077837289899438, max_delta_step=None,
             max_depth=4, min_child_weight=3, missing=nan,
             monotone_constraints=None, n_estimators=4374, n_jobs=-1,
             num_parallel_tree=None, random_state=42,
             reg_alpha=0.009577332554566264, reg_lambda=1.2196486740226578,
             scale_pos_weight=None, subsample=0.75, tree_method=None,
             validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106830 (STD 0.0120)
Raw CV RMSE 18770.443039 (STD 2229.3542)


## BayesOpt

In [21]:
# bayesopt
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = BayesOptSearch(utility_kwargs={
    "kind": "ucb",
    "kappa": 2.5,
    "xi": 0.0
})

# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.quniform(0, 6, 1),
        'min_child_weight': tune.quniform(0, 6, 1),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_bayesopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,iter,total time (s),mse
my_xgb_831b14e6,TERMINATED,,0.330905,0.0951207,4.39196,3.59195,1644.58,0.156079,5.80846,0.833088,1,49.1084,0.116099
my_xgb_831c1cd8,TERMINATED,,0.500836,0.0710992,0.123507,5.81946,8341.18,0.212418,18.1826,0.491702,1,150.897,0.114678
my_xgb_831d1598,TERMINATED,,0.278182,0.0529509,2.59167,1.74737,6157.34,0.13958,29.2145,0.583181,1,130.728,0.116462
my_xgb_831e1592,TERMINATED,,0.392052,0.0787324,1.19804,3.08541,5964.9,0.0465458,60.7545,0.485262,1,124.774,0.117542
my_xgb_831f083a,TERMINATED,,0.0987887,0.0949397,5.79379,4.85038,3115.68,0.0977623,68.4233,0.620076,1,51.8592,0.116519
my_xgb_83200956,TERMINATED,,0.141529,0.0500225,0.206331,5.45592,2661.92,0.662556,31.1712,0.660034,1,25.1061,0.110889
my_xgb_8320f0b4,TERMINATED,,0.460033,0.0193006,5.81751,4.6508,9401.04,0.894838,59.79,0.860937,1,312.447,0.114999
my_xgb_8321e172,TERMINATED,,0.116369,0.0204023,0.271364,1.95198,3947.91,0.271422,82.8738,0.578377,1,35.4733,0.110967
my_xgb_8322c5d8,TERMINATED,,0.260701,0.0547269,0.845545,4.81318,838.051,0.986888,77.2245,0.499358,1,11.9215,0.120372
my_xgb_8323c726,TERMINATED,,0.0541416,0.0817307,4.24114,4.37404,7735.58,0.0741372,35.8466,0.457935,1,79.7085,0.117329


Start Time           2020-10-13 03:59:37.514701
End Time             2020-10-13 07:02:42.296129
3:03:04


In [22]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df

,mse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.min_child_weight,config.subsample,config.colsample_bytree,config.reg_alpha,config.reg_lambda,config.gamma,config.learning_rate
trial_id,,,,,,,,,,,,
42d43826,0.108465,2020-10-13_04-47-44,161.605700,2459,3,2.178384,0.728034,0.335998,0.129373,1.306109,0,0.017106
41f4a6fc,0.108571,2020-10-13_04-25-06,120.924064,2453,2,1.884766,0.646968,0.083735,0.349982,20.428522,0,0.032042
42c9af6e,0.108979,2020-10-13_04-45-10,118.100566,2458,2,4.637463,0.597422,0.111506,0.444802,5.230673,0,0.032992
47bcaf58,0.109267,2020-10-13_06-27-11,107.539983,2663,3,2.955540,0.818149,0.067963,0.763891,33.911951,0,0.043589
42f4732a,0.109345,2020-10-13_04-47-30,123.946153,2458,3,1.971131,0.677033,0.082519,0.746090,11.048329,0,0.031264
...,...,...,...,...,...,...,...,...,...,...,...,...
3f5d484a,0.125266,2020-10-13_04-07-34,45.837948,3951,2,3.985822,0.574333,0.244892,0.873355,80.473413,0,0.003945
46ecc036,0.128449,2020-10-13_06-15-09,349.789933,2455,6,2.692943,0.400781,0.517892,0.389601,24.936141,0,0.003405
3fca69ca,0.148612,2020-10-13_04-08-14,30.920428,2456,4,2.370457,0.651065,0.199336,0.791412,18.272218,0,0.002384


In [23]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3359983713603832,
             gamma=0, gpu_id=None, importance_type='gain',
             interaction_constraints=None, learning_rate=0.01710560084665138,
             max_delta_step=None, max_depth=3,
             min_child_weight=2.1783844345363184, missing=nan,
             monotone_constraints=None, n_estimators=2459, n_jobs=-1,
             num_parallel_tree=None, random_state=42,
             reg_alpha=0.1293725288758705, reg_lambda=1.3061085139687396,
             scale_pos_weight=None, subsample=0.7280344096788107,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.108465 (STD 0.0124)
Raw CV RMSE 19074.202762 (STD 2297.0290)


## Optuna

In [9]:
# optuna
NUM_SAMPLES=256
optuna_xgb = my_xgb

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = OptunaSearch()
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.quniform(0, 6, 1),
        'min_child_weight': tune.quniform(0, 6, 1),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True,
            "name": get_random_tag(6)
        }           
    }
}

analysis = tune.run(optuna_xgb,
                    name="xgb_optuna",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),                    
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,iter,total time (s),mse
my_xgb_7edc71ca,TERMINATED,,0.4,0.0549176,6,5,971.932,0.160481,4.33692,0.55,1,47.854,0.113531
my_xgb_7edf04a8,TERMINATED,,0.4,0.00372368,3,3,6717.95,0.173497,0.943211,0.8,1,208.262,0.109689
my_xgb_7ee16dc4,TERMINATED,,0.8,0.00234011,2,6,4611.82,0.056709,64.3363,0.85,1,166.827,0.123827
my_xgb_7ee4424c,TERMINATED,,0.6,0.0391407,6,0,951.779,0.931624,0.000193777,0.55,1,59.6289,0.11494
my_xgb_7ee6da34,TERMINATED,,0.2,0.0290001,1,5,6807.64,0.000147101,0.0125031,0.75,1,92.2411,0.107897
my_xgb_7ee807ec,TERMINATED,,0.35,0.00513378,5,2,205.679,0.0101822,38.1721,0.75,1,2.36608,4.19202
my_xgb_7eea6a00,TERMINATED,,0.4,0.0220675,4,6,400.957,0.004395,0.00265938,0.7,1,15.6655,0.110964
my_xgb_7eed08a0,TERMINATED,,0.65,0.00120357,6,6,767.648,0.0129051,0.000774433,0.8,1,47.0648,4.5792
my_xgb_7eee2172,TERMINATED,,0.7,0.00136,6,5,2227.7,0.734805,0.0319066,0.9,1,67.2932,0.578644
my_xgb_7ef14a82,TERMINATED,,0.4,0.001821,6,0,183.462,0.00013567,0.85854,0.65,1,2.25279,8.26642


Start Time           2020-10-13 14:22:14.974528
End Time             2020-10-13 14:59:32.357208
0:37:17


In [10]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df

,mse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.min_child_weight,config.subsample,config.colsample_bytree,config.reg_alpha,config.reg_lambda,config.gamma,config.learning_rate
trial_id,,,,,,,,,,,,
7f953a84,0.106348,2020-10-13_14-42-07,25.841348,2800,4,2.0,0.70,0.05,0.003014,0.038068,0,0.017830
7fcbfc18,0.107188,2020-10-13_14-48-02,66.311119,3534,3,2.0,0.80,0.35,0.006353,0.001028,0,0.014852
7ffe364c,0.107684,2020-10-13_14-53-19,14.275375,535,3,3.0,0.70,0.55,0.022284,0.003241,0,0.074685
7f4d9bfc,0.107824,2020-10-13_14-32-36,38.333613,4813,3,5.0,0.70,0.05,0.185167,7.454486,0,0.011166
7ee6da34,0.107897,2020-10-13_14-23-52,92.241096,6807,3,5.0,0.75,0.20,0.000147,0.012503,0,0.029000
...,...,...,...,...,...,...,...,...,...,...,...,...
7f6323d2,9.393934,2020-10-13_14-33-13,1.413051,137,2,6.0,0.45,0.10,0.702820,0.010945,0,0.001495
7fe7b2fa,9.411590,2020-10-13_14-51-04,9.253910,177,5,0.0,0.90,0.75,0.002673,0.001806,0,0.001146
7fe06504,9.468593,2020-10-13_14-49-47,1.184400,110,3,1.0,0.40,0.05,0.080022,0.002307,0,0.001789


In [11]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.05, gamma=0, gpu_id=None,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.017830307668061096, max_delta_step=None,
             max_depth=4, min_child_weight=2.0, missing=nan,
             monotone_constraints=None, n_estimators=2800, n_jobs=-1,
             num_parallel_tree=None, random_state=42,
             reg_alpha=0.0030137080970469286, reg_lambda=0.0380680484151218,
             scale_pos_weight=None, subsample=0.7000000000000001,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106348 (STD 0.0114)
Raw CV RMSE 18679.843445 (STD 2113.9369)


# LightGBM with Optuna

In [12]:
# tune LightGBM
print("LightGBM")
#!conda install -y -c conda-forge lightgbm

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

def my_lgbm(config):
    
    # fix these configs 
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    config['num_leaves'] = 2 + int(config['num_leaves'])
    
    lgbm = LGBMRegressor(objective='regression',
                         num_leaves=config['num_leaves'],
                         learning_rate=config['learning_rate'],
                         n_estimators=config['n_estimators'],
                         max_bin=200,
                         bagging_fraction=config['bagging_fraction'],
                         feature_fraction=config['feature_fraction'],
                         feature_fraction_seed=7,
                         min_data_in_leaf=2,
                         verbose=-1,
                         # early stopping params, maybe in fit
                         #early_stopping_rounds=early_stopping_rounds,
                         #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                         #num_boost_round=num_boost_round,
                         )
    
    scores = np.sqrt(-cross_val_score(lgbm, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


Trial name,status,loc,bagging_fraction,feature_fraction,learning_rate,n_estimators,num_leaves,wandb/api_key_file,wandb/log_config,wandb/project,iter,total time (s),mse
my_lgbm_8c1cee7c,TERMINATED,,0.645556,0.262363,0.00137437,4626.62,9,secrets/wandb.txt,True,iowa,1,38.5115,0.113285
my_lgbm_8c1e78e6,TERMINATED,,0.701044,0.259482,0.0734068,2853.98,7,secrets/wandb.txt,True,iowa,1,17.2909,0.109697
my_lgbm_8c1fe924,TERMINATED,,0.668672,0.484699,0.0188581,1180.42,6,secrets/wandb.txt,True,iowa,1,10.5364,0.111762
my_lgbm_8c213b30,TERMINATED,,0.540469,0.0265392,0.01101,191.949,0,secrets/wandb.txt,True,iowa,1,0.404755,0.2627
my_lgbm_8c22a204,TERMINATED,,0.528971,0.0457216,0.091503,905.561,6,secrets/wandb.txt,True,iowa,1,2.51982,0.110973
my_lgbm_8c240cb6,TERMINATED,,0.781842,0.599419,0.00253267,494.528,5,secrets/wandb.txt,True,iowa,1,5.25374,0.194386
my_lgbm_8c25869a,TERMINATED,,0.794835,0.45121,0.0400602,7345.14,3,secrets/wandb.txt,True,iowa,1,39.3354,0.111324
my_lgbm_8c26f264,TERMINATED,,0.616283,0.541449,0.0027847,395.202,6,secrets/wandb.txt,True,iowa,1,4.36902,0.203936
my_lgbm_8c28977c,TERMINATED,,0.748052,0.787677,0.00167074,185.032,8,secrets/wandb.txt,True,iowa,1,3.56561,0.313964
my_lgbm_8c2a170a,TERMINATED,,0.688821,0.361669,0.00933932,897.568,8,secrets/wandb.txt,True,iowa,1,8.33463,0.113955


Start Time           2020-10-13 15:05:35.820524
End Time             2020-10-13 15:16:14.678302
0:10:38


In [15]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.num_leaves', 'config.bagging_fraction',
       'config.feature_fraction', 'config.learning_rate']].sort_values('mse')
analysis_results_df

,mse,date,time_this_iter_s,config.n_estimators,config.num_leaves,config.bagging_fraction,config.feature_fraction,config.learning_rate
trial_id,,,,,,,,
8ca842c4,0.105969,2020-10-13_15-07-38,6.322035,2758,5,0.754804,0.049949,0.027701
8cb900aa,0.106189,2020-10-13_15-07-46,6.636419,1742,10,0.517380,0.055519,0.020289
8cf74d92,0.106252,2020-10-13_15-08-40,6.396669,2715,5,0.700356,0.043747,0.026954
8d5cf14c,0.106657,2020-10-13_15-09-44,4.099934,1758,5,0.699876,0.039295,0.045055
8e200510,0.106706,2020-10-13_15-11-50,4.707111,1734,5,0.712548,0.047620,0.043218
...,...,...,...,...,...,...,...,...
8d1512be,0.323659,2020-10-13_15-08-45,1.353710,247,11,0.724945,0.102919,0.001350
8d8c8dda,0.344946,2020-10-13_15-10-02,1.187111,153,11,0.725653,0.170354,0.001309
8e78f3e6,0.346311,2020-10-13_15-12-22,2.775580,155,11,0.681775,0.646190,0.001111


In [18]:
num_leaves = analysis_results_df.iloc[0]['config.num_leaves']
bagging_fraction = analysis_results_df.iloc[0]['config.bagging_fraction']
feature_fraction = analysis_results_df.iloc[0]['config.feature_fraction']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'num_leaves': num_leaves,
    'bagging_fraction': bagging_fraction,
    'feature_fraction': feature_fraction,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

lgbm = LGBMRegressor(objective='regression',
                     num_leaves=best_config['num_leaves'],
                     learning_rate=best_config['learning_rate'],
                     n_estimators=best_config['n_estimators'],
                     max_bin=200,
                     bagging_fraction=best_config['bagging_fraction'],
                     feature_fraction=best_config['feature_fraction'],
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbose=-1,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

LGBMRegressor(bagging_fraction=0.7548044328777966,
              feature_fraction=0.04994854048380451, feature_fraction_seed=7,
              learning_rate=0.027701226718815856, max_bin=200,
              min_data_in_leaf=2, n_estimators=2758, num_leaves=5,
              objective='regression', verbose=-1)

Log1p CV RMSE 0.105969 (STD 0.0126)
Raw CV RMSE 18612.385681 (STD 2325.3205)


# Ray Cluster

- Look at `ray1.0.yaml`
- Edit `ray1.0.yaml` file with your region, availability zone, subnet, imageid information
    - to get those variables launch the latest Deep Learning AMI (Ubuntu 18.04) Version 35.0 into a small instance in your favorite region/zone
    - test that it works
    - note those 4 variables: region, availability zone, subnet, AMI imageid
    - terminate the instance and edit `ray1.0.yaml` accordingly
    - in future you can create your own image with everything pre-installed and specify its AMI imageid, instead of using the generic image and installing everything at launch.
- To run the cluster: 
`ray up ray1.0.yaml`
    - Creates head instance using image specified.
    - Installs ray and related requirements
    - Clones this Iowa repo
    - Launches worker nodes per auto-scaling parameters (currently we fix the number of nodes because we're not benching the time the cluster will take to auto-scale)
- After cluster starts you can check AWS console and note that several instances launched.
- Check `ray monitor ray1.0.yaml` for any error messages
- Run Jupyter on the cluster with port forwarding
 `ray exec ray1.0.yaml --port-forward=8899 'source ~/anaconda3/bin/activate tensorflow_p36 && jupyter notebook --port=8899'`
- Open the notebook on the generated URL e.g. http://localhost:8899/?token=5f46d4355ae7174524ba71f30ef3f0633a20b19a204b93b4
- Make sure to hoose the default kernel to make sure it runs in the conda environment with all installs
- Make sure to use the ray.init() command given in the startup messages.
- You can also run a terminal on the head node of the cluster with
 `ray attach /Users/drucev/projects/iowa/ray1.0.yaml`
- You can also ssh explicitly with the IP address and the generated private key
 `ssh -o IdentitiesOnly=yes -i ~/.ssh/ray-autoscaler_1_us-east-1.pem ubuntu@54.161.200.54`
- run port forwarding to the Ray dashboard with   
`ray dashboard ray1.0.yaml`
and then open
 http://localhost:8265/

see https://docs.ray.io/en/latest/cluster/launcher.html for additional info

In [ ]:
# make sure local ray is shutdown
ray.shutdown()


In [ ]:
# launch cluster in terminal
# initialize ray on cluster
ray.init(address='localhost:6379', _redis_password='5241590000000000')


In [ ]:
# run hyperopt/xgb on cluster
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.randint(0, 6),
        'min_child_weight': tune.randint(0, 6),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1, 
                    raise_on_failed_trial=False, # otherwise no reults df returned if any trial error
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df


In [ ]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


In [ ]:
#lgbm on cluster
print("LightGBM")

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    raise_on_failed_trial=False, # otherwise no reults df returned if any trial error                    
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.num_leaves', 'config.bagging_fraction',
       'config.feature_fraction', 'config.learning_rate']].sort_values('mse')
analysis_results_df


In [ ]:
num_leaves = analysis_results_df.iloc[0]['config.num_leaves']
bagging_fraction = analysis_results_df.iloc[0]['config.bagging_fraction']
feature_fraction = analysis_results_df.iloc[0]['config.feature_fraction']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'num_leaves': num_leaves,
    'bagging_fraction': bagging_fraction,
    'feature_fraction': feature_fraction,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

lgbm = LGBMRegressor(objective='regression',
                     num_leaves=best_config['num_leaves'],
                     learning_rate=best_config['learning_rate'],
                     n_estimators=best_config['n_estimators'],
                     max_bin=200,
                     bagging_fraction=best_config['bagging_fraction'],
                     feature_fraction=best_config['feature_fraction'],
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbose=-1,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

## Concluding thoughts

We observe a modest but non-negligeable improvement in the target metric with a less manual process vs. sequential tuning.

For the complex algos, I think my days of grid search are over, and I'll be using HyperOpt and Optuna for hyperparameter optimization going forward. In every case I've applied them, I've gotten at least a small improvement in the best metrics I found using grid search methods.

These two algorithms seem to be the most popular but I may try the other algos systematically. 
I was surprised that Elasticnet, i.e. regularized linear regression outperforms boosting. 
This dataset has been heavily engineered so that linear methods work well. Predictors were chosen using lasso/elasticnet and we used log and Box-Cox transforms to force predictors to follow assumptions of least-squares.  

This tends to validate one of the [critiques of machine learning](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3624052), that the most powerful ML methods don't necessarily converge all the way to the best solution. If you have a ground truth that is linear plus noise, a complex XGBoost or neural network algorithm should get arbitrarily close to a closed-form solution but will never match it exactly due to complexity of the algorithm. XGBoost is piecewise constant and the complex neural network is subject to the vagaries of stochastic gradient descent. 

But Elasticnet with L1 + L2 regularization plus gradient descent and hyperparameter optimization is still machine learning, simply the form best matched to the problem. In the real world where things don't match assumptions of least-squares, boosting performs extremely well. And even on this dataset, engineered for the linear models, ensembling Elasticnet with XGBoost, LightGBM, SVR, neural networks works best of all. 